In [1]:
!pip3 install redis


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import feedparser
import newspaper
from newspaper import Article
from datetime import datetime

c:\Python310\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (None)/charset_normalizer (2.1.1) doesn't match a supported version!
  warnings.warn(


Required libraries


In [3]:
rss_fields=["http://rss.cnn.com/rss/cnn_topstories.rss", "http://qz.com/feed", "http://feeds.foxnews.com/foxnews/politics", "http://feeds.reuters.com/reuters/businessNews", "http://feeds.feedburner.com/NewshourWorld", "https://feeds.bbci.co.uk/news/world/asia/india/rss.xml"]

In [4]:
def parse_feed(feed_url):
    feed=feedparser.parse(feed_url)
    articles=[]
    
    for entry in feed.entries:
        articles ={
          'title': entry.title,
            'link': entry.link,
            'published': entry.published if 'published' in entry else None,
            'source': feed_url  
        }
        
        try:
            news_article = Article(entry.link)
            news_article.download()
            news_article.parse()
            article = {}
            article['content'] = news_article.text
        except Exception as e:
            article['content'] = None  
        articles.update(article)
    
    return articles
    

In [ ]:
articles_fields = []
for field in rss_fields:
    article = parse_feed(field)
    articles_fields.append(article)
    print(field)
    
print(arcticles_fields)

In [27]:
print(articles_fields)

[{'title': "Review: 'Barry' takes a whack at its farewell season", 'link': 'https://www.cnn.com/2023/04/14/entertainment/barry-season-4-review/index.html', 'published': 'Fri, 14 Apr 2023 12:33:08 GMT', 'source': 'http://rss.cnn.com/rss/cnn_topstories.rss', 'content': 'CNN —\n\n“Barry” has taken chances from the very beginning, which is certainly true of a fourth and final season that picks up where the third left off, with its hitman-turned-wannabe actor getting arrested. That paves the way for an even darker season that accentuates the show’s ensemble aspect while leaning a little too heavily on blurring lines with flights of fancy.\n\nThanks to “Succession,” “Barry” won’t be the highest-profile goodbye on HBO this spring, but the Emmy-nominated series isn’t chopped liver either. It’s fair to say, in fact, that while these episodes don’t quite measure up to what’s gone before, even a less-lethal “Barry” is still very, very good.\n\nBill Hader’s auteur turn as director-producer-star re

feedparser to collect news aricles


In [28]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('news_articles.db')
cursor = conn.cursor()

# Create a table to store articles
cursor.execute('''CREATE TABLE IF NOT EXISTS articles (
                  id INTEGER PRIMARY KEY AUTOINCREMENT,
                  title TEXT,
                  content TEXT,
                  published DATETIME,
                  source TEXT,
                  url TEXT UNIQUE,
                  category TEXT)''')

# Function to store articles in the database
def store_article(article, category):
    try:
        cursor.execute('''INSERT INTO articles (title, content, published, source, url, category) 
                          VALUES (?, ?, ?, ?, ?, ?)''', 
                          (article['title'], article['content'], article['published'], article['source'], article['link'], category))
        conn.commit()
    except sqlite3.IntegrityError:
        print(f"Duplicate article: {article['title']}")


database storage


In [29]:
import nltk
from nltk.corpus import stopwords

# Example categories with keywords
categories = {
    'Politics': ['election', 'government', 'politician', 'policy'],
    'Technology': ['tech', 'AI', 'software', 'hardware', 'innovation'],
    'Health': ['health', 'medicine', 'doctor', 'virus', 'vaccine']
}

# Categorization function based on keywords
def categorize_article(content):
    if not content:
        return 'Uncategorized'
    
    content_words = set(content.lower().split())
    for category, keywords in categories.items():
        if any(keyword in content_words for keyword in keywords):
            return category
    return 'Uncategorized'


article categorization

In [ ]:
from celery import Celery

# Create a new Celery instance
app = Celery('news_collector', broker='redis://localhost:6379/0')

# You can configure other Celery options if needed
app.conf.update(
    task_routes={
        'news_collector.*': {'queue': 'news'}
    },
    task_serializer='json',
    result_serializer='json',
    accept_content=['json'],
    timezone='UTC',
    enable_utc=True
)
# from your_parser_script import parse_feed, store_article, categorize_article

app = Celery('news_collector', broker='redis://localhost:6379/0')

@app.task
def process_article(article):
    category = categorize_article(article['content'])
    store_article(article, category)

# Queueing articles for processing
for feed in rss_fields:
    articles = parse_feed(feed)
    for article in articles:
        process_article.delay(article)  # Asynchronous task


task queueing with celery and redis

In [ ]:
import logging

logging.basicConfig(filename='news_collector.log', level=logging.INFO)

def store_article(article, category):
    try:
        cursor.execute('''INSERT INTO articles (title, content, published, source, url, category) 
                          VALUES (?, ?, ?, ?, ?, ?)''', 
                          (article['title'], article['content'], article['published'], article['source'], article['link'], category))
        conn.commit()
        logging.info(f"Article stored: {article['title']}")
    except sqlite3.IntegrityError:
        logging.warning(f"Duplicate article: {article['title']}")
    except Exception as e:
        logging.error(f"Error storing article: {e}")


logging and error handling

In [ ]:
celery -A news_collector worker --loglevel=info


run redis in backgrounud,Start celery

application running

In [ ]:
Python Code: Includes feed parsing, database management, categorization, task queueing, and logging.

Database Dump: To export data from SQLite to CSV:

sqlite3 news_articles.db ".mode csv" ".headers on" ".output articles.csv" "SELECT * FROM articles;"